In [1]:
import pandas as pd

In [2]:
FEATURES_TRAINTEST = pd.read_csv('/home/docker_current/datasets/VK/FINAL_FEATURES_TRAINTEST.tsv', sep = '\t')
TARGETS_DATES_TRAINTEST = pd.read_csv('/home/docker_current/datasets/VK/FINAL_TARGETS_DATES_TRAINTEST.tsv', sep = '\t')
FEATURES_TRAINTEST.shape

(159288, 1015)

In [3]:
traintest_df = TARGETS_DATES_TRAINTEST.merge(FEATURES_TRAINTEST, how='inner')
del TARGETS_DATES_TRAINTEST, FEATURES_TRAINTEST
traintest_df.shape

(159288, 1017)

In [4]:
selective = True
import pickle

with open('important_feat.pickle', 'rb') as handle:
    selected_feat = pickle.load(handle)

selected_feat = selected_feat + ['RETRO_DT', 'TARGET', 'CLIENT_ID']

if selective:
    traintest_df = traintest_df[selected_feat]

traintest_df.shape

(159288, 203)

In [5]:
GNN_features = pd.read_csv('GNN_feat_1.csv')
GNN_features.shape

(159288, 129)

In [6]:
traintest_df = traintest_df.merge(GNN_features, how='inner')

In [7]:
traintest_df.shape

(159288, 331)

### Загружаем дату, чтобы правильно сматчить айдишники для валидации

In [8]:
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split
import numpy as np
import torch

x_all = np.load('/home/docker_current/datasets/graph/nodes.npy', allow_pickle=True)
y_all = np.load('/home/docker_current/datasets/graph/lables.npy', allow_pickle=True)
date = np.load('/home/docker_current/datasets/graph/dates.npy', allow_pickle=True).astype(int)
edges = np.load('/home/docker_current/datasets/graph/edges.npy', allow_pickle=True)
weights = np.load('/home/docker_current/datasets/graph/weights.npy', allow_pickle=True)

y_all[y_all == 'test'] = 2
y_all = y_all.astype(int)

x = torch.tensor(x_all).float()
y = torch.tensor(y_all)
edges = torch.tensor(edges)

date = torch.tensor(date)
weights = torch.tensor(weights)

data = Data(x=x, edge_index=edges, y=y, weights=weights)


idx = np.arange(len(data.x))
idx = idx[(data.y != -1) & (data.y != 2)]
train_idx, val_idx = train_test_split(idx, test_size=0.2, stratify=data.y[(data.y != -1) & (data.y != 2)], random_state=42)

data.train_idx = torch.tensor(train_idx)
data.val_idx = torch.tensor(val_idx)

In [9]:
idx_to_client = np.load('idx_to_client.npy')
train_client_id = idx_to_client[train_idx]
val_client_id = idx_to_client[val_idx]

In [10]:
train_df = traintest_df[traintest_df['CLIENT_ID'].isin(train_client_id)]
val_df = traintest_df[traintest_df['CLIENT_ID'].isin(val_client_id)]

In [11]:
X_train = train_df.drop(['RETRO_DT', 'TARGET', 'CLIENT_ID'], axis=1).values
y_train = train_df['TARGET'].values

X_val = val_df.drop(['RETRO_DT', 'TARGET', 'CLIENT_ID'], axis=1).values
y_val = val_df['TARGET'].values

In [12]:
from catboost import CatBoostClassifier

# model = CatBoostClassifier(
#         iterations=500,
#         learning_rate=0.01,
#         silent=False,
#         random_seed=42,
#         task_type='GPU',
#         eval_metric = 'AUC',
#         max_depth=10,
#     )

model = CatBoostClassifier(iterations=2000,
                          task_type="GPU",
                           devices='0:1',
                           eval_metric = 'AUC')

model.fit(X_train,
          y_train,
          eval_set=[(
        X_val, 
        y_val)],
          verbose=True,
          plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.036073
0:	test: 0.5614099	best: 0.5614099 (0)	total: 11.7ms	remaining: 23.3s
1:	total: 20.7ms	remaining: 20.7s
2:	total: 29ms	remaining: 19.3s
3:	total: 38.4ms	remaining: 19.1s


Default metric period is 5 because AUC is/are not implemented for GPU


4:	total: 48.8ms	remaining: 19.5s
5:	test: 0.5795136	best: 0.5795136 (5)	total: 61.1ms	remaining: 20.3s
6:	total: 71.6ms	remaining: 20.4s
7:	total: 80.8ms	remaining: 20.1s
8:	total: 90.4ms	remaining: 20s
9:	total: 105ms	remaining: 20.8s
10:	test: 0.5820742	best: 0.5820742 (10)	total: 117ms	remaining: 21.2s
11:	total: 128ms	remaining: 21.2s
12:	total: 138ms	remaining: 21.1s
13:	total: 148ms	remaining: 21s
14:	total: 159ms	remaining: 21s
15:	test: 0.5824911	best: 0.5824911 (15)	total: 171ms	remaining: 21.3s
16:	total: 182ms	remaining: 21.2s
17:	total: 193ms	remaining: 21.3s
18:	total: 204ms	remaining: 21.2s
19:	total: 214ms	remaining: 21.2s
20:	test: 0.5850886	best: 0.5850886 (20)	total: 227ms	remaining: 21.4s
21:	total: 238ms	remaining: 21.4s
22:	total: 247ms	remaining: 21.2s
23:	total: 256ms	remaining: 21.1s
24:	total: 264ms	remaining: 20.9s
25:	test: 0.5849380	best: 0.5850886 (20)	total: 276ms	remaining: 20.9s
26:	total: 286ms	remaining: 20.9s
27:	total: 296ms	remaining: 20.8s
28:	tot

In [13]:
X_test = traintest_df[traintest_df['TARGET'] == 'test'].drop(['RETRO_DT', 'TARGET', 'CLIENT_ID'], axis=1).values
test_id = traintest_df[traintest_df['TARGET'] == 'test']['CLIENT_ID']

y_pred = model.predict_proba(X_test)[:, 1]

In [14]:
submit_df = pd.DataFrame(test_id.values, columns=['CLIENT_ID'])
submit_df['TARGET1'] = y_pred

In [15]:
sample_submit = pd.read_csv('sample_submit.csv')
submit = sample_submit.merge(submit_df, how='inner')
submit['TARGET'] = submit['TARGET1']

submit = submit.drop(['TARGET1'], axis=1)
submit.to_csv('GNN_true.csv', index=False)

In [19]:
submit

,CLIENT_ID,TARGET
0,1504364,0.297100
1,1592092,0.135289
2,1582159,0.374388
3,1431753,0.201158
4,1571196,0.324901
...,...,...
31853,1518155,0.186465
31854,1544546,0.135274
31855,1584674,0.245658
31856,1733854,0.195634


In [16]:
# check = pd.read_csv('GNN_true.csv')
# check

In [17]:
sample_submit

,CLIENT_ID,TARGET
0,1504364,0.282197
1,1592092,0.241287
2,1582159,0.276877
3,1431753,0.241196
4,1571196,0.273023
...,...,...
31853,1518155,0.207584
31854,1544546,0.159157
31855,1584674,0.344294
31856,1733854,0.242887


In [20]:
sample_submit

,CLIENT_ID,TARGET
0,1504364,0.282197
1,1592092,0.241287
2,1582159,0.276877
3,1431753,0.241196
4,1571196,0.273023
...,...,...
31853,1518155,0.207584
31854,1544546,0.159157
31855,1584674,0.344294
31856,1733854,0.242887
